## Data visualization

In [1]:
#import the nescessary libraries
import pandas as pd
import numpy as np

In [2]:
#Import the cleaned data set
data_path = 'C:/Users/User/Desktop/dfrev2.csv'
df1 = pd.read_csv(data_path)
df1.tail()

FileNotFoundError: [Errno 2] File C:/Users/User/Desktop/dfrev2.csv does not exist: 'C:/Users/User/Desktop/dfrev2.csv'

### Task 1: 
For the summer month (i.e., June, August, or September) with most trips, visualize the number of started trips per PLZ region (you’ll have to find geo data for that yourselves!) in a map.

The task requires geodata in order to visualize the number of trips by regions. The geodata available for this task is available on https://www.suche-postleitzahl.org/downloads.
The original file for administrative areas by postal code was available in .kml format. The file was then converted into geojson format for this project.

In [5]:
#Import the geojson data
import json
with open('C:/Users/User/Desktop/ffpostcode.geojson') as json_data:
    ffgeo = json.load(json_data)
ffgeo

{'type': 'FeatureCollection',
 'name': 'PLZ 5-stellig Stadt Frankfurt am Main',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'Name': '60306',
    'description': '60306 Frankfurt am Main, Opernturm'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[8.6695498, 50.1158427],
      [8.6699022, 50.116553],
      [8.6706714, 50.1163693],
      [8.6705891, 50.1156601],
      [8.6707255, 50.1156328],
      [8.6707175, 50.1155649],
      [8.6695498, 50.1158427]]]}},
  {'type': 'Feature',
   'properties': {'Name': '60308', 'description': '60308 Frankfurt'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[8.6520947, 50.1121949],
      [8.6526051, 50.112719],
      [8.653367, 50.1124064],
      [8.6527681, 50.1118609],
      [8.6520947, 50.1121949]]]}},
  {'type': 'Feature',
   'properties': {'Name': '60310',
    'description': '60310 Frankfurt am Main (Taunusturm)'},
   'geometry': {'type'

The information we need is in the features keys. We can check the first data to see how it looks.
Also, the zipcode is in the key 'Name' in the 'properties' tab

In [6]:
ffgeo['features'][0]

{'type': 'Feature',
 'properties': {'Name': '60306',
  'description': '60306 Frankfurt am Main, Opernturm'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[8.6695498, 50.1158427],
    [8.6699022, 50.116553],
    [8.6706714, 50.1163693],
    [8.6705891, 50.1156601],
    [8.6707255, 50.1156328],
    [8.6707175, 50.1155649],
    [8.6695498, 50.1158427]]]}}

In [8]:
#Create a new column called count in the base data frame. Assign each row value 1.
#Use the method groupby to group the data by zip_codes and calculate the sum of the count colum.
#Add the results to a new data frame with 2 columns 'Zipcodes' and 'Total trips'
df1['count'] = 1 
startbyzip = pd.DataFrame(df1[['Zip_codes', 'count']].groupby(['Zip_codes']).sum().reset_index().values.tolist(), columns = ['Zipcodes','Total_trips'])
startbyzip.tail()

Zipcodes  Total_trips
36     63067          856
37     65929          443
38     65933          340
39     65934           35
40     65936           18

This notebook uses the plotly library to visualize the cholorepth map

In [3]:
pip install plotly

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=ed2f562b06e629494efdef29bd635334c95d470946a35b249c90e1d5506669ca
  Stored in directory: c:\users\manue\appdata\local\pip\cache\wheels\f9\8d\8d\f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [38]:
#Use the plotly lib to visualize the map
#data for ploting is the startbyzip data frame, using the total trips for data and zipcodes for location
#featureidkey indicates the location of the region, the information is stored in the Name key of the properties tab
import plotly.express as px
fig = px.choropleth_mapbox(startbyzip, geojson=ffgeo, color='Total_trips',
                           locations='Zipcodes', featureidkey='properties.Name',
                           center={'lat': 50.107254, 'lon': 8.685223},
                           mapbox_style="carto-positron", zoom=10)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title='Choropleth map of total trips by postal areas')
fig.show()

<center>Choropleth map of total trips by postal areas<center>

In [24]:
import datetime as dt
df1['Start_Time'] = pd.to_datetime(df1['Start_Time'], format="%Y-%m-%d %H:%M:%S")
df1['Month'] = df1['Start_Time'].dt.month
df1.tail()

Unnamed: 0  Bike_number          Start_Time  Start_Latitude  \
72754      154338        44115 2019-08-09 03:50:00       50.091681   
72755      154339        44115 2019-08-10 22:52:00       50.091681   
72756      154340        44115 2019-08-11 22:58:00       50.091681   
72757      154343         6208 2019-10-05 23:21:00       50.107599   
72758      154344         6208 2019-10-06 23:27:00       50.107599   

       Start_Longitude  Start_Station  Weekday  End_Latitude  End_Longitude  \
72754         8.653257         4206.0     True     50.091681       8.653257   
72755         8.653257         4206.0    False     50.091681       8.653257   
72756         8.653257         4206.0    False     50.091681       8.653257   
72757         8.664168         4240.0    False     50.107599       8.664168   
72758         8.664168         4240.0    False     50.107599       8.664168   

       Duration  End_Station_number  Zip_codes  Bikes_on_position  count  \
72754       5.0              4206.0      60528                  5      1   
72755       7.0              4206.0      60528                  5      1   
72756       7.0              4206.0      60528                  4      1   
72757       7.0              4240.0      60329                 13      1   
72758       7.0              4240.0      60329                 14      1   

       Month  
72754      8  
72755      8  
72756      8  
72757     10  
72758     10

In [28]:
df2 = df1.set_index('Start_Time')

In [30]:
df_jun = df2['2019-06-01 00:00:00':'2019-06-30 23:59:59']
df_aug = df2['2019-08-01 00:00:00':'2019-08-31 23:59:59']
df_sep = df2['2019-09-01 00:00:00':'2019-09-30 23:59:59']

In [31]:
df_jun.tail()

Unnamed: 0  Bike_number  Start_Latitude  Start_Longitude  \
Start_Time                                                                      
2019-06-29 12:59:00      154323        43400       50.134167         8.768889   
2019-06-09 21:03:00      154327        43643       50.096199         8.660890   
2019-06-10 04:20:00      154329        43643       50.096199         8.660890   
2019-06-11 01:20:00      154331        43643       50.096199         8.660890   
2019-06-11 06:46:00      154332        43643       50.096199         8.660890   

                     Start_Station  Weekday  End_Latitude  End_Longitude  \
Start_Time                                                                 
2019-06-29 12:59:00        42003.0    False     50.134167       8.768889   
2019-06-09 21:03:00         4278.0    False     50.096199       8.660890   
2019-06-10 04:20:00         4278.0     True     50.096199       8.660890   
2019-06-11 01:20:00         4278.0     True     50.096199       8.660890   
2019-06-11 06:46:00         4278.0     True     50.096199       8.660890   

                     Duration  End_Station_number  Zip_codes  \
Start_Time                                                     
2019-06-29 12:59:00      15.0             42003.0      60386   
2019-06-09 21:03:00       7.0              4278.0      60528   
2019-06-10 04:20:00       5.0              4278.0      60528   
2019-06-11 01:20:00       7.0              4278.0      60528   
2019-06-11 06:46:00       7.0              4278.0      60528   

                     Bikes_on_position  count  Month  
Start_Time                                            
2019-06-29 12:59:00                  4      1      6  
2019-06-09 21:03:00                  5      1      6  
2019-06-10 04:20:00                  5      1      6  
2019-06-11 01:20:00                  5      1      6  
2019-06-11 06:46:00                  5      1      6

In [32]:
startbyzip_jun = pd.DataFrame(df_jun[['Zip_codes', 'count']].groupby(['Zip_codes']).sum().reset_index().values.tolist(), columns = ['Zipcodes','Total_trips'])
startbyzip_aug = pd.DataFrame(df_aug[['Zip_codes', 'count']].groupby(['Zip_codes']).sum().reset_index().values.tolist(), columns = ['Zipcodes','Total_trips'])
startbyzip_sep = pd.DataFrame(df_sep[['Zip_codes', 'count']].groupby(['Zip_codes']).sum().reset_index().values.tolist(), columns = ['Zipcodes','Total_trips'])

In [33]:
startbyzip_jun

Zipcodes  Total_trips
0      60308           63
1      60311          634
2      60313          809
3      60314          552
4      60316          463
5      60318          787
6      60320          384
7      60322          582
8      60323          536
9      60325          626
10     60326          302
11     60327          459
12     60329         1173
13     60385          425
14     60386           46
15     60388            4
16     60389           80
17     60431           28
18     60433            8
19     60435           16
20     60437            1
21     60438            4
22     60439           70
23     60486          567
24     60487          439
25     60488          106
26     60489           24
27     60528          392
28     60529            1
29     60594         1024
30     60596          272
31     60598          143
32     60599           20
33     63067          191
34     65929           71
35     65933           42
36     65934            2
37     65936            2

In [39]:
fig = px.choropleth_mapbox(startbyzip_jun, geojson=ffgeo, color='Total_trips',
                           locations='Zipcodes', featureidkey='properties.Name',
                           center={'lat': 50.107254, 'lon': 8.685223},
                           mapbox_style="carto-positron", zoom=10)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [40]:
fig = px.choropleth_mapbox(startbyzip_aug, geojson=ffgeo, color='Total_trips',
                           locations='Zipcodes', featureidkey='properties.Name',
                           center={'lat': 50.107254, 'lon': 8.685223},
                           mapbox_style="carto-positron", zoom=10)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [41]:
fig = px.choropleth_mapbox(startbyzip_sep, geojson=ffgeo, color='Total_trips',
                           locations='Zipcodes', featureidkey='properties.Name',
                           center={'lat': 50.107254, 'lon': 8.685223},
                           mapbox_style="carto-positron", zoom=10)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Task 3: 
Create a heatmap based on an interesting aspect of the data, e.g., end locations of trips shortly before the start of a major public event.